In [45]:
import os, sys, json

In [46]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["HADOOP_INSTALL"] = "/home/hadoop/hadoop"
os.environ["HADOOP_HOME"] = os.environ["HADOOP_INSTALL"]
os.environ["HADOOP_MAPRED_HOME"] = os.environ["HADOOP_INSTALL"]
os.environ["HADOOP_COMMON_HOME"] = os.environ["HADOOP_INSTALL"]
os.environ["HADOOP_HDFS_HOME"] = os.environ["HADOOP_INSTALL"]
os.environ["HADOOP_YARN_HOME"] = os.environ["HADOOP_INSTALL"]
os.environ["HADOOP_CONF_DIR"] = os.path.join(os.environ["HADOOP_INSTALL"], "/etc/hadoop")
os.environ["SPARK_HOME"] = "/home/hadoop/spark"
sys.path.insert(0, os.path.join(os.environ["SPARK_HOME"], "python"))
sys.path.append(os.path.join(os.environ["SPARK_HOME"], "python/lib/py4j-0.10.9.2-src.zip"))

In [47]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("HelloLines") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "1024M") \
    .getOrCreate()
sc = spark.sparkContext
print(sc.uiWebUrl)

http://192.168.121.62:4044


In [48]:
def getDataFromHDFS(filePath):
   try:
      return spark.read.csv(filePath)
   except Exception as e:
      return -1

In [49]:
rdd = sc.textFile("hdfs:/datasets/spotify/tracks.json")

In [50]:
rdd2 = rdd.map(lambda x: json.loads(x))
dfp = rdd2.toDF()
dfp

DataFrame[album_name: string, album_uri: string, artist_name: string, artist_uri: string, duration_ms: bigint, pid: bigint, pos: bigint, track_name: string, track_uri: string]

1. Statistics about songs duration

1.1 - Generate a table containing the minimum, average and maximum duration, in milliseconds, of the songs in the dataset.

In [51]:
aggregates = getDataFromHDFS('q1.1.csv')
if aggregates == -1:
    min = dfp.agg({ 'duration_ms' : 'min' })
    avg = dfp.agg({ 'duration_ms' : 'avg' })
    max = dfp.agg({ 'duration_ms' : 'max' })
    aggregates = min.join(avg)
    aggregates = aggregates.join(max)
    aggregates.write.csv('./q1.1.csv')
aggregates.show()

+---+------------------+--------+
|_c0|               _c1|     _c2|
+---+------------------+--------+
|  0|234408.54976216817|10435467|
+---+------------------+--------+



1.2 - Compute the first and third quartiles (denoted Q1​ and Q3​), as well as the interquartile range (IRQ) (Q3​−Q1​).

In [52]:
quantiles = dfp.approxQuantile("duration_ms", [0.25, 0.75], 0)
q1, q3 = quantiles[0], quantiles[1]
q1, q3

(198333.0, 258834.0)

1.3 - Compute the set of songs with durations that are not outliers, as defined by the IQRR methodology. In other words, identify all songs with duration xx such that Q1 − 1.5 × IQR < x < Q3 + 1.5 × IQR.

In [77]:
from pyspark.sql.functions import col

def iqr_outlier_treatment(df, columns, factor=1.5):
    """
    Detects and treats outliers using IQR for multiple variables in a PySpark DataFrame.

    :param dataframe: The input PySpark DataFrame
    :param columns: A list of columns to apply IQR outlier treatment
    :param factor: The IQR factor to use for detecting outliers (default is 1.5)
    :return: The processed DataFrame with outliers treated
    """

    iqr = q3 - q1

    # Define the upper and lower bounds for outliers
    lower_bound = q1 - factor * iqr
    upper_bound = q3 + factor * iqr
    
    for column in columns:
        # Filter outliers and no outliers
        df_no_outliers = df.filter((col(column) >= lower_bound) & (col(column) <= upper_bound))
        df_outliers = df.filter((col(column) <= lower_bound) & (col(column) >= upper_bound))
        #df_no_outliers, df_outliers

    return df_no_outliers, df_outliers

In [78]:
# not_outliers
iqr_outlier_treatment(dfp, dfp.columns)
#df_treated
#df_no_outliers = df_treated[0]

#df_no_outliers.count()

#df_no_outliers

# aggregates = getDataFromHDFS('q1.3.csv')
# if aggregates == -1:
#     min = df_no_outliers.agg({ 'duration_ms' : 'min' })
#     avg = df_no_outliers.agg({ 'duration_ms' : 'avg' })
#     max = df_no_outliers.agg({ 'duration_ms' : 'max' })
#     aggregates = min.join(avg)
#     aggregates = aggregates.join(max)
#     aggregates.write.csv('./q1.3.csv')
# aggregates.show()

(DataFrame[album_name: string, album_uri: string, artist_name: string, artist_uri: string, duration_ms: bigint, pid: bigint, pos: bigint, track_name: string, track_uri: string],
 DataFrame[album_name: string, album_uri: string, artist_name: string, artist_uri: string, duration_ms: bigint, pid: bigint, pos: bigint, track_name: string, track_uri: string])

1.4 - Using the IQRR methodology, how many songs would be considered outliers and removed from analysis? Generate a new table containing the minimum, average and maximum duration of the remaining songs.

In [ ]:
# outliers
df_outliers = df_treated[1]

df_outliers.count()

aggregates = getDataFromHDFS('q1.4.csv')
if aggregates == -1:
    min = df_outliers.agg({ 'duration_ms' : 'min' })
    avg = df_outliers.agg({ 'duration_ms' : 'avg' })
    max = df_outliers.agg({ 'duration_ms' : 'max' })
    aggregates = min.join(avg)
    aggregates = aggregates.join(max)
    aggregates.write.csv('./q1.4.csv')
aggregates.show()

3. Playlists's behavior

What is more common, playlists where there are many songs by the same artist or playlists with more diverse songs?

In [ ]:
# playlists com mais musicas do mesmo artista



# playlists com mais musicas de artistas variados
